In [ ]:
!pip install QuantumRingsLib
!pip install quantumrings-toolkit-qiskit
!pip install qiskit==1.4.1
!pip install matplotlib

In [3]:
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import DraperQFTAdder, PhaseEstimation, QFT
from qiskit.visualization import plot_histogram
import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2
from quantumrings.toolkit.qiskit import QrJobV1
from IPython.display import clear_output
from fractions import Fraction
from math import log10, ceil
import random
import time
import json

In [38]:
#sign up for quantum rings at https://portal.quantumrings.com/signup/ and generate an API toke.
#then fill out beow fields
QUANTUM_RINGS_TOKEN = "[INPUT TOKEN HERE]"
QUANTUM_RINGS_EMAIL = "[INPUT EMAIL HERE]"

if QUANTUM_RINGS_TOKEN == "[INPUT EMAIL HERE]" or QUANTUM_RINGS_EMAIL == "[INPUT EMAIL HERE]":
  raise ValueError("Please sign up on QuantumRings at https://portal.quantumrings.com/signup/, generate an API token, and set the above variables to your token and account email.")

In [4]:
#essential helping functions
def euler_gcd(a, b):
  while a != 0:
    a, b = b % a, a
  return b

def toBinary(n, bits):
  binary = []
  for x in range(bits):
    binary.append(n % 2)
    n //= 2
  binary.reverse()
  return binary

def fromBinary(n):
  #this can take list of ints or a bitstring
  sum = 0
  for x in n:
    sum = sum * 2 + int(x)
  return sum

def getGoodCoprime(n):
  a = random.randint(2, n-1)
  if euler_gcd(a, n) != 1:
    return getGoodCoprime(n)
  return a

In [5]:
#--QUANTUM FUNCTIONS--#
def QPE(a, N):
  working_reg = QuantumRegister(bits, name='working')
  control_reg = QuantumRegister(bits, name='control')

  qc = QuantumCircuit(working_reg, control_reg, name="QPE")

  n = len(control_reg)
  for i in range(n):
    power=2**i
    a_power=pow(a,power, N)
    for j in range(len(working_reg)):
      if(a_power>>j)&1:
        qc.cx(working_reg[i],control_reg[j])

  return qc

def build_shors_circuit(a, n):
  working_reg = QuantumRegister(bits, name='working')
  control_reg = QuantumRegister(bits, name='control')
  out_reg = ClassicalRegister(bits, name='out')

  qc = QuantumCircuit(working_reg, control_reg, out_reg)

  qc.h(working_reg)
  qc.h(control_reg)

  qc.append(QPE(a, n).to_gate(), [*working_reg, *control_reg])
  qc.append(QFT(bits, do_swaps=False).inverse(), control_reg)

  qc.measure(working_reg, out_reg)

  return qc

def run(qc, shots):
  qr_provider = QuantumRingsProvider(token =QUANTUM_RINGS_TOKEN, name=QUANTUM_RINGS_EMAIL)
  mybackend = QrBackendV2(qr_provider, num_qubits = qc.num_qubits)
  qc_transpiled = transpile(qc, mybackend, initial_layout=[i for i in range(0, qc.num_qubits)])
  job = mybackend.run(qc_transpiled, shots = shots)
  result = job.result()
  return result.get_counts()

In [6]:
#--CLASSICAL POSTPROCESSING--#
def shors_postprocess(resultVal, a):
  r = Fraction(resultVal, 2**bits).limit_denominator(n).denominator # N = the number you're factoring

  if r % 2 == 0:
    x = pow(a, int(r/2),n) #the third argument changes the way exponentiation is done here
                           #it does a^(r/2) mod n, which does not modify the results in the end
                           #while keeping the values from overflowing
    val1 = euler_gcd(n, x-1)
    val2 = euler_gcd(n, x+1)

    if (val1 != 1 and val1 != n):
      return (val1, n/val1)
    elif (val2 != 1 and val1 != n):
      return (val2, n/val2)
  return False #failed post-process

In [ ]:
#--VARIABLES--#
n = int(input("Enter Semiprime to factor: "))
bits = ceil(log10(n) / log10(2))
if bits > 100:
  raise ValueError(f"Semiprime to factor must be < 2^100, recieved {n}")
else:
  print(f"Using {2*bits} total qubits")

benchmark = False #if true, benchmarking occurs
                  #program runs 15 iterations of Shor's algo for N
                  #and prints to the console the avg time and avg attempts

In [ ]:
%matplotlib inline
#--TYING IT ALL TOGETHER--#
def runOneShor(n):
  attempts = 0
  found_factors = False
  start = time.time()

  while found_factors == False:
    attempts += 1
    a = getGoodCoprime(n)
    clear_output(wait=True)
    print(f"Attempt {attempts} - trying a={a}")
    counts = run(build_shors_circuit(a, n), 1)
    resultVal = fromBinary(list(counts.keys())[0])
    found_factors = shors_postprocess(resultVal, a)
  print(f"Factors found: {found_factors}")
  return (attempts, time.time() - start, found_factors)

def benchmark15(n):
  average_attempts = 0
  average_time = 0

  for i in range(15):
    data = runOneShor(n)
    average_attempts += data[0]
    average_time += data[1]
  average_attempts /= 15
  average_time /= 15
  return average_attempts, average_time

if benchmark:
  x, y = benchmark15(n)
  print(f"Benchmark Data for N={n}")
  print(f"\tAverage Attempts: {x}")
  print(f"\tAverage Time: {y}s")
else:
  data = runOneShor(n)
  print(f"Total Attempts: {data[0]}")
  print(f"Time: {data[1]}s")